In [1]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Estatico':
    
    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]<>'OUTROS'
    ORDER BY [Data de Faturamento]
    
    """,
    
    'Aberto':
    
    """
    
    SELECT * FROM netfeira.vw_aberto
    WHERE [Data do Pedido]=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)  
    
    """,
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    WHERE Fabricante LIKE '%MCCAIN%'
    
    """,
    
    'Calendario':
    
    """

    SELECT * FROM netfeira.vw_calend
    WHERE YEAR([Data])=YEAR(GETDATE()) AND MONTH([Data])=MONTH(GETDATE()) AND [Dia Útil]=1
    
    """
    
}

In [3]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [4]:
metas_df=pd.DataFrame()

metas_df=tabelas_df['Produto'][['Grupo MCCAIN','Produto']].groupby(['Grupo MCCAIN'],as_index=False).count()

metas_df.loc[metas_df['Grupo MCCAIN'].str.contains('FOOD'),'Meta KG']=(80*1000)

metas_df.loc[metas_df['Grupo MCCAIN'].str.contains('VAREJO'),'Meta KG']=(40*1000)

metas_df

,Grupo MCCAIN,Produto,Meta KG
0,MCCAIN FOOD,57,80000.0
1,MCCAIN VAREJO,27,40000.0


In [5]:
vendas_df=pd.DataFrame()

vendas_df=tabelas_df['Estatico']

In [6]:
vendas_df=vendas_df[['SKU','Total Venda','Peso Líquido KG','Peso Bruto KG']].groupby(['SKU'],as_index=False).sum()

In [7]:
vendas_df.columns

Index(['SKU', 'Total Venda', 'Peso Líquido KG', 'Peso Bruto KG'], dtype='object')

In [8]:
vendas_df=vendas_df.merge(tabelas_df['Produto'],on='SKU',how='inner')[['SKU','Produto','Fabricante','Grupo MCCAIN','Categoria','Linha','Total Venda', 'Peso Líquido KG', 'Peso Bruto KG']]

In [9]:
aberto_df=pd.DataFrame()

aberto_df=tabelas_df['Aberto'].loc[tabelas_df['Aberto']['Situação']=='AB']

In [10]:
aberto_df=aberto_df[['SKU','Total Venda','Peso Líquido KG','Peso Bruto KG']].groupby(['SKU'],as_index=False).sum()

In [11]:
aberto_df=aberto_df.merge(tabelas_df['Produto'],on='SKU',how='inner')[['SKU','Produto','Fabricante','Grupo MCCAIN','Categoria','Linha','Total Venda', 'Peso Líquido KG', 'Peso Bruto KG']]

In [12]:
dia_df=pd.DataFrame()

dia_df=tabelas_df['Aberto']

dia_df=dia_df[['SKU','Total Venda','Peso Líquido KG','Peso Bruto KG']].groupby(['SKU'],as_index=False).sum()

dia_df=dia_df.merge(tabelas_df['Produto'],on='SKU',how='inner')[['SKU','Produto','Fabricante','Grupo MCCAIN','Categoria','Linha','Total Venda', 'Peso Líquido KG', 'Peso Bruto KG']]

In [13]:
metas_df['Faturado KG']=metas_df['Grupo MCCAIN'].apply(lambda info: vendas_df['Peso Líquido KG'].loc[vendas_df['Grupo MCCAIN']==info].sum())

In [14]:
metas_df['Total KG']=metas_df['Grupo MCCAIN'].apply(lambda info: dia_df['Peso Líquido KG'].loc[dia_df['Grupo MCCAIN']==info].sum())

In [15]:
metas_df['Aberto KG']=metas_df['Grupo MCCAIN'].apply(lambda info: aberto_df['Peso Líquido KG'].loc[aberto_df['Grupo MCCAIN']==info].sum())

In [16]:
calend_df=pd.DataFrame()

calend_df=tabelas_df['Calendario']

util=len(calend_df['Data'].unique().tolist())

trabalhado=len(calend_df['Data Trabalhada'].loc[~calend_df['Data Trabalhada'].isnull()].unique().tolist())-1

restante=util-trabalhado

In [17]:
metas_df['Realizado KG']=metas_df['Faturado KG']+metas_df['Aberto KG']

In [18]:
metas_df['Projeção KG']=metas_df['Realizado KG'].apply(lambda info: round((info/trabalhado)*util,2))

In [19]:
metas_df['Meta %']=round(metas_df['Realizado KG']/metas_df['Meta KG'],4)*100

In [20]:
metas_df['Projeção %']=round(metas_df['Projeção KG']/metas_df['Meta KG'],4)*100

In [21]:
metas_df['Diário KG']=round(metas_df['Meta KG']/util,2)

In [24]:
metas_df['Diário %']=round(metas_df['Total KG']/metas_df['Diário KG'],4)*100

In [26]:
metas_df['Meta (-)']=metas_df['Realizado KG']-metas_df['Meta KG']

In [28]:
metas_df['Diário (-)']=metas_df['Total KG']-metas_df['Diário KG']

In [30]:
metas_df['Projeção (-)']=metas_df['Projeção KG']-metas_df['Meta KG']

In [31]:
metas_df

,Grupo MCCAIN,Produto,Meta KG,Faturado KG,Total KG,Aberto KG,Realizado KG,Projeção KG,Meta %,Projeção %,Diário KG,Diário %,Meta (-),Diário (-),Projeção (-)
0,MCCAIN FOOD,57,80000.0,28255.27,130.00,130.00,28385.27,49674.22,35.48,62.09,3809.52,3.41,-51614.73,-3679.52,-30325.78
1,MCCAIN VAREJO,27,40000.0,18229.66,242.24,242.24,18471.90,32325.83,46.18,80.81,1904.76,12.72,-21528.10,-1662.52,-7674.17
